In [2]:
import requests
import pandas as pd 


In [3]:
df = pd.read_csv("2024-2025-data.csv")
print(df.shape)
df.head()

(118, 67)


,Department,7/1/2024,7/8/2024,7/15/2024,7/22/2024,7/29/2024,8/5/2024,8/12/2024,8/19/2024,8/26/2024,...,8/25/2025,9/1/2025,9/8/2025,9/15/2025,9/22/2025,9/29/2025,10/6/2025,10/13/2025,10/20/2025,10/27/2025
0,Accountancy,25,22,17,17,18,12,4,6,45,...,53,173,217,259,266,231,218,236,249,200
1,Advertising,4,1,4,2,2,0,3,2,36,...,41,148,172,229,304,278,290,320,270,282
2,Aerospace Engineering,12,13,7,10,5,5,5,7,59,...,103,133,120,146,180,193,183,186,116,104
3,African American Studies,1,0,0,0,0,0,0,1,2,...,0,0,1,1,2,0,0,1,3,3
4,Ag Ldrshp Educ Comm Program,1,1,3,1,1,1,1,1,4,...,7,4,3,12,5,7,7,9,3,6


In [4]:
print(df["Department"].unique().size)
df["sum"] = df.sum(axis = 1, numeric_only = True) 
df_sum = df[["Department", "sum"]]
df_sum

118


,Department,sum
0,Accountancy,7829
1,Advertising,8249
2,Aerospace Engineering,4414
3,African American Studies,60
4,Ag Ldrshp Educ Comm Program,449
...,...,...
113,Translation & Interpreting St,3
114,Undergraduate Admissions,60
115,Urban & Regional Planning,1319
116,Vet Clinical Medicine,2


In [5]:
df_colleges = pd.read_csv("degree-college.csv")
df_colleges.columns = ["Degree", "College"]
df_colleges

,Degree,College
0,Adult Development,ACES
1,Agri-Accounting,ACES
2,Agribusiness Markets & Management,ACES
3,Agribusiness and Sustainable Food Production E...,ACES
4,Agricultural & Applied Economics,ACES
...,...,...
507,Pathobiology,VETMED
508,Veterinary Medical Science - Comparative Biosc...,VETMED
509,Veterinary Medical Science - Pathobiology,VETMED
510,Veterinary Medical Sciences - Veterinary Clini...,VETMED


In [6]:
import re 
def clean_text(s: str) -> str: 
    if pd.isna(s) : 
        return ""
    s = s.lower()
    s = s.replace("&", "and")
    s = re.sub(r'[^a-z0-9\s]', " ", s)
    s = re.sub(r'\s+', " ", s).strip()
    return s


In [7]:
df["cleaned_department"] = df["Department"].apply(clean_text)
df_colleges["cleaned_degree"] = df_colleges["Degree"].apply(clean_text)


In [8]:
from rapidfuzz import process, fuzz

degree_choices = df_colleges["Degree"].unique().tolist()

def best_degree_match(cleaned_department: str, cutoff: int = 80): 
    if not cleaned_department: 
        return None
    match = process.extractOne(
        cleaned_department, 
        degree_choices, 
        scorer = fuzz.token_set_ratio, 
        score_cutoff = cutoff
    )
    if match is None: 
        return None
    # tuple unpacking for output of match; "_" gets the index value which doesn't matter
    best_str, best_score, _ = match
    return best_str
df["degree_college_match"] = df["cleaned_department"].apply(best_degree_match)


In [9]:
pd.set_option('display.max_rows', None)
df[["cleaned_department", "degree_college_match"]]
#pd.reset_option('display.max_rows')

,cleaned_department,degree_college_match
0,accountancy,Accountancy
1,advertising,Advertising
2,aerospace engineering,Aerospace Engineering
3,african american studies,African American Studies
4,ag ldrshp educ comm program,None
5,agr and consumer economics,None
6,agr consumer and env sci admn,None
7,agr consumer and env sciences,None
8,agricultural and biological engr,None
9,animal sciences,Animal Sciences
